# Configurables

In [2]:
# Configurable values that may or may not need to be changed per run
dir_location = 'internal' # Location that this run is on: internal, external, or cloud (i.e. OneDrive)
output_to_cloud = False # Overrides the output path to always output to the cloud (i.e. OneDrive)
is_original_data = False # Original data is the dataset used by the authors of ParticleSeg3D, False results in the DigiM tablet dataset
weights_tag = 'original_particle_seg' # Name of the weights that will be used
run_tag = 'pretrained_initial_tablet' # Name of the run, like what name will be used to label the outputs generated from this run
conda_env = 'Senior_Design_py310' # Name of conda environment containing all necessary libraries, packages, and modules

print('Values configured')

Values configured


# Set Paths

In [3]:
import os

# Set path to the base directory
if dir_location.lower() == 'internal':
    base_path = r'C:\Senior_Design'
elif dir_location.lower() == 'external':
    base_path = r'D:\Senior_Design'
elif dir_location.lower() == 'cloud':
    base_path = r'C:\Users\dchen\OneDrive - University of Connecticut\Courses\Year 4\Fall 2024\BME 4900 and 4910W (Kumavor)\Python\Files'
else:
    raise ValueError('Invalid directory location type')

# Set base paths to input, output, and weights        
base_input_path = os.path.join(base_path, 'database')
base_output_path = os.path.join(base_path, 'outputs')
if output_to_cloud:
    base_output_path = os.path.join(r'C:\Users\dchen\OneDrive - University of Connecticut\Courses\Year 4\Fall 2024\BME 4900 and 4910W (Kumavor)\Python\Files', 'outputs')
base_weights_path = os.path.join(base_path, 'weights')

# Set Zarr (model output) and TIFF (readable output) paths
output_zarr_path = os.path.join(base_output_path, 'zarr', run_tag)
output_tiff_path = os.path.join(base_output_path, 'tiff', run_tag)

# Set dataset path
if is_original_data:
    input_path = os.path.join(base_input_path, 'orignal_dataset', 'grayscale', 'dataset')
else:
    input_path = os.path.join(base_input_path, 'tablet_dataset', 'grayscale', 'dataset')

# Set weights path
weights_path = os.path.join(base_weights_path, weights_tag)

# Ensure paths with spaces or parentheses are quoted (for using the directory in OneDrive, does not matter if there are no spaces or parentheses)
input_path = f'"{input_path}"'
output_zarr_path = f'"{output_zarr_path}"'
output_tiff_path = f'"{output_tiff_path}"'
weights_path = f'"{weights_path}"'

print('Paths set')

Paths set


# Check CUDA Availability

In [4]:
import torch
torch.cuda.is_available()

True

# Run ParticleSeg3D Using Subprocess Module

## Run ParticleSeg3D on All Tablets Without Progress Bar

In [5]:
## No real time updates
# import subprocess

# # Define the commands to execute in Anaconda PowerShell Prompt
# activate_env = 'conda activate ' + conda_env
# inference = 'ps3d_inference -i ' + input_path + ' -o ' + output_zarr_path + ' -m ' + weights_path
# command = activate_env + '; ' + inference

# # Run the commands
# process = subprocess.run(
#     ['powershell', '-Command', command], 
#     stdout=subprocess.PIPE, 
#     stderr=subprocess.PIPE, 
#     text=True
# )

# # Output the results
# print("STDOUT:", process.stdout)
# print("STDERR:", process.stderr)

## Run ParticleSeg3D on All Tablets with Progress Bar

In [6]:
# # Runs inferences on all tablets

# import subprocess
# import sys

# # Define the commands to execute in Anaconda PowerShell Prompt
# activate_env = 'conda activate ' + conda_env
# inference = 'ps3d_inference -i ' + input_path + ' -o ' + output_zarr_path + ' -m ' + weights_path
# command = activate_env + '; ' + inference

# # Run the command with unbuffered output
# process = subprocess.Popen(
#     ['powershell', '-Command', command], 
#     stdout=subprocess.PIPE, 
#     stderr=subprocess.PIPE, 
#     text=True,
#     bufsize=1  # Line-buffered for real-time updates
# )

# print("Running inference...\n")

# # Read output in real-time and ensure TQDM updates properly
# for line in iter(process.stdout.readline, ''):
#     sys.stdout.write(line)  # Ensures immediate output
#     sys.stdout.flush()

# # Capture any errors
# process.stdout.close()
# stderr_output = process.stderr.read()
# process.wait()

# if stderr_output:
#     print("\nSTDERR:", stderr_output)

# print("\nInference Completed!")

## Run ParticleSeg3D on Each Tablet in Different Cells With Progress Bar

In [13]:
# Runs inferences on 1_Microsphere

import subprocess
import sys
import os

# Define the commands to execute in Anaconda PowerShell Prompt
input_path = os.path.join(os.path.dirname(input_path).strip('"'), '1_Microsphere')
input_path = f'"{input_path}"'
activate_env = 'conda activate ' + conda_env
inference = 'ps3d_inference -i ' + input_path + ' -o ' + output_zarr_path + ' -m ' + weights_path
command = activate_env + '; ' + inference

# Run the command with unbuffered output
process = subprocess.Popen(
    ['powershell', '-Command', command], 
    stdout=subprocess.PIPE, 
    stderr=subprocess.PIPE, 
    text=True,
    bufsize=1  # Line-buffered for real-time updates
)

print("Running inference...\n")

# Read output in real-time and ensure TQDM updates properly
for line in iter(process.stdout.readline, ''):
    sys.stdout.write(line)  # Ensures immediate output
    sys.stdout.flush()

# Capture any errors
process.stdout.close()
stderr_output = process.stderr.read()
process.wait()

if stderr_output:
    print("\nSTDERR:", stderr_output)

print("\nInference Completed!")

Running inference...


STDERR: The string is missing the terminator: ".
    + CategoryInfo          : ParserError: (:) [], ParentContainsErrorRecordException
    + FullyQualifiedErrorId : TerminatorExpectedAtEndOfString
 


Inference Completed!


In [ ]:
# Runs inferences on 2_Tablet

import subprocess
import sys
import os

# Define the commands to execute in Anaconda PowerShell Prompt
input_path = os.path.join(os.path.dirname(input_path).strip('"'), '2_Tablet')
input_path = f'"{input_path}"'
activate_env = 'conda activate ' + conda_env
inference = 'ps3d_inference -i ' + input_path + ' -o ' + output_zarr_path + ' -m ' + weights_path
command = activate_env + '; ' + inference

# Run the command with unbuffered output
process = subprocess.Popen(
    ['powershell', '-Command', command], 
    stdout=subprocess.PIPE, 
    stderr=subprocess.PIPE, 
    text=True,
    bufsize=1  # Line-buffered for real-time updates
)

print("Running inference...\n")

# Read output in real-time and ensure TQDM updates properly
for line in iter(process.stdout.readline, ''):
    sys.stdout.write(line)  # Ensures immediate output
    sys.stdout.flush()

# Capture any errors
process.stdout.close()
stderr_output = process.stderr.read()
process.wait()

if stderr_output:
    print("\nSTDERR:", stderr_output)

print("\nInference Completed!")

In [ ]:
# Runs inferences on 3_SprayDriedDispersion

import subprocess
import sys
import os

# Define the commands to execute in Anaconda PowerShell Prompt
input_path = os.path.join(os.path.dirname(input_path).strip('"'), '3_SprayDriedDispersion')
input_path = f'"{input_path}"'
activate_env = 'conda activate ' + conda_env
inference = 'ps3d_inference -i ' + input_path + ' -o ' + output_zarr_path + ' -m ' + weights_path
command = activate_env + '; ' + inference

# Run the command with unbuffered output
process = subprocess.Popen(
    ['powershell', '-Command', command], 
    stdout=subprocess.PIPE, 
    stderr=subprocess.PIPE, 
    text=True,
    bufsize=1  # Line-buffered for real-time updates
)

print("Running inference...\n")

# Read output in real-time and ensure TQDM updates properly
for line in iter(process.stdout.readline, ''):
    sys.stdout.write(line)  # Ensures immediate output
    sys.stdout.flush()

# Capture any errors
process.stdout.close()
stderr_output = process.stderr.read()
process.wait()

if stderr_output:
    print("\nSTDERR:", stderr_output)

print("\nInference Completed!")

"C:\Senior_Design\database\tablet_dataset\grayscale\3_SprayDriedDispersion"
Running inference...

Samples:  ['3_SprayDriedDispersion']
Starting inference of sample:  3_SprayDriedDispersion

Predicting: 0it [00:00, ?it/s]
Predicting:   0%|          | 0/122 [00:00<?, ?it/s]


# Convert Zarr Output to TIFF

In [3]:
import numpy as np
import os
import zarr
import tifffile
import subprocess

def safe_makedirs(path):
    """
    Safely create directories. If it fails due to invalid characters (e.g., quotes),
    it strips invalid characters and retries.
    """
    try:
        os.makedirs(path, exist_ok=True)
    except OSError as e:
        print(f"Error creating directory {path}: {e}")
        # Retry after stripping quotes
        cleaned_path = path.strip('"')
        print(f"Retrying with cleaned path: {cleaned_path}")
        try:
            os.makedirs(cleaned_path, exist_ok=True)
            path = cleaned_path
        except OSError as e2:
            print(f"Failed again with cleaned path {cleaned_path}: {e2}")
            raise e2  # Reraise the error if it still fails
        print()
    return path


def clean_path(path):
    """
    Strips invalid characters (e.g., quotes) from the path.
    """
    return path.strip('"')


def is_valid_zarr_directory(zarr_dir):
    """
    Validates the Zarr directory to ensure it exists and contains the required structure.
    Returns True if valid, otherwise False.
    """
    if not os.path.exists(zarr_dir):
        print(f"Error: Zarr directory {zarr_dir} does not exist.")
        print()
        return False

    if not os.path.isdir(zarr_dir):
        print(f"Error: {zarr_dir} is not a directory.")
        print()
        return False

    # Check if the directory contains valid subdirectories with `.zarr` files
    for image_name in os.listdir(zarr_dir):
        image_path = os.path.join(zarr_dir, image_name)
        zarr_file = os.path.join(image_path, f"{image_name}.zarr")
        if os.path.isdir(image_path) and os.path.exists(zarr_file):
            continue  # Valid image directory with a .zarr file
        else:
            print(f"Warning: {image_path} is missing a corresponding .zarr file.")
            print()
            return False

    return True


def convert_zarr_to_tiff(zarr_dir, tiff_dir):
    """
    Converts Zarr files to TIFF format, placing them directly in the `.tiff` folder.
    """
    print(f"Original Zarr Directory: {zarr_dir}")
    print(f"Original TIFF Directory: {tiff_dir}")
    print()

    # Try validating the Zarr directory
    try:
        # Attempt to list files in the Zarr directory
        os.listdir(zarr_dir)
    except OSError as e:
        print(f"Error accessing Zarr directory {zarr_dir}: {e}")
        # Clean path and retry
        zarr_dir = clean_path(zarr_dir)  # Update zarr_dir with the cleaned path
        print(f"Retrying with cleaned Zarr Directory: {zarr_dir}")
        # Check again with the cleaned path
        if not is_valid_zarr_directory(zarr_dir):
            print("Invalid Zarr directory after cleaning. Aborting conversion.")
            print()
            return
        print()

    # Ensure the output base directory exists for TIFF, and create the `.tiff` folder
    tiff_dir = safe_makedirs(tiff_dir)

    # Iterate through each folder in the Zarr directory
    for image_name in os.listdir(zarr_dir):
        image_path = os.path.join(zarr_dir, image_name)
        if os.path.isdir(image_path):
            zarr_input = os.path.join(image_path, f"{image_name}.zarr")

            print(f"Processing image: {image_name}")
            print(f"Zarr Input: {zarr_input}")
            print(f"TIFF Output Directory: {tiff_dir}")
            print()
            
            # Open Zarr dataset
            image_zarr = zarr.open(zarr_input, mode='r')

            # Convert Zarr data to uint8 (ensure it's within the valid range for uint8)
            if image_zarr.dtype != np.uint8:
                print("Converting data to float32...")
                for i in range(image_zarr.shape[0]):
                    image_slice = image_zarr[i]

                    # Normalize and scale the data to the uint8 range (0-255)
                    image_slice = (image_slice / np.max(image_slice) * 255).astype(np.uint8) # Converts the segmentation labels into visualizable grayscale colors
                    # image_slice = np.clip(image_slice, 0, 255)  # Clip values to 0-255
                    # image_slice = image_slice.astype(np.uint8)   # Convert to uint8
                    # image_slice = image_slice.astype(np.float32)   # Convert to float32

                    # Save the slice as a TIFF image
                    tiff_output_dir = os.path.join(tiff_dir, image_name)
                    tiff_output_dir = safe_makedirs(tiff_output_dir)
                    tifffile.imwrite(os.path.join(tiff_output_dir, f"{image_name}_{i:04d}.tiff"), image_slice)
            else:
                try:
                    # Run the external command to convert Zarr to TIFF (does not seem to work, the generated TIFF images cannot be read...)
                    command = ["ps3d_zarr2tiff", "-i", zarr_input, "-o", tiff_dir]
                    result = subprocess.run(command, check=True, capture_output=True, text=True)
                    print(f"Converted {zarr_input} to {tiff_dir}")
                    print()
                    print("Standard Output:", result.stdout)
                    print("Standard Error:", result.stderr)
                except subprocess.CalledProcessError as e:
                    print(f"Error converting {zarr_input} to TIFF: {e}")
                    print()
                except Exception as e:
                    print(f"Unexpected error while processing {image_name}: {e}")
                    print()

In [6]:
# Convert Zarr files to TIFF
convert_zarr_to_tiff(output_zarr_path, output_tiff_path)

Original Zarr Directory: "C:\Senior_Design\outputs\zarr\pretrained_initial_tablet"
Original TIFF Directory: "C:\Senior_Design\outputs\tiff\pretrained_initial_tablet"

Error accessing Zarr directory "C:\Senior_Design\outputs\zarr\pretrained_initial_tablet": [WinError 123] The filename, directory name, or volume label syntax is incorrect: '"C:\\Senior_Design\\outputs\\zarr\\pretrained_initial_tablet"'
Retrying with cleaned Zarr Directory: C:\Senior_Design\outputs\zarr\pretrained_initial_tablet

Invalid Zarr directory after cleaning. Aborting conversion.

